In [1]:
# import packages
import numpy as np
import pylab as py
import pdb
import matplotlib.pyplot as plt
import math

# import spisea packages
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity

# import nbody6tools reader to read data
from nbody6tools import Reader

# import nbody62spisea to retrieve star ages
from nbody62spisea import converter

/home/wyz5rge/.local/lib/python3.9/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /scratch/wyz5rge/models/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


## Set up parameters, create isochrone grid

In [2]:
# define isochrone parameters
AKs = 0
dist = 410
evo_model = evolution.Baraffe15()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()
filt_list = ['jwst,F115W', 'jwst,F182M']
filters = ['m_jwst_F115W', 'm_jwst_F182M']
iso_dir = 'isochrones/'
metallicity = 0

In [3]:
# read nbody data
path_to_data = '../../../../../../storage/jfarias/Data/Orion/massive_clusters/M3000new/sigma0p1/fiducial/sfe_ff001/00/'
snapshot = Reader.read_snapshot(path_to_data, time = 20)

# convert snapshot values from nbody units to astrophysical, extract masses
snapshot.to_physical()

# read ages for simulated stars
cluster_table = converter.to_spicea_table(snapshot)

print(cluster_table)

        mass        isMultiple        compMass             systemMass              e                 log_a               age        
------------------- ---------- ---------------------- ------------------- -------------------- ------------------ ------------------
0.06895842179945424        0.0                     [] 0.06895842179945424                  nan                nan  18.62630555946738
0.08626887187757219        0.0                     [] 0.08626887187757219                  nan                nan  3.069082513027311
  2.326448478984193        0.0                     []   2.326448478984193                  nan                nan  18.33038253724003
0.04093112166717806        0.0                     [] 0.04093112166717806                  nan                nan  3.069082513027311
                ...        ...                    ...                 ...                  ...                ...                ...
0.03156546631062232        1.0 [0.017415375068988628] 0.0489808413796

In [4]:
masses = cluster_table['mass']
level_ages = cluster_table['age']

for i in range(len(level_ages)):
    level_ages[i] = level_ages[i] * 1e6

# convert log ages from log(Myr) to log(yr)
log_ages = np.log10(level_ages)

# print(log_ages)
print("Min log age = " + str(min(log_ages)))
print("Max log age = " + str(max(log_ages)))

# print(level_ages)
print("Min level age = " + str(min(level_ages)))
print("Max level age = " + str(max(level_ages)))

Min log age = 6.407048843194058
Max log age = 7.3018036073193215
Min level age = 2552988.4099163027
Max level age = 20035657.88269043


In [5]:
# find index of first binary, create array of companion masses
first_binary = 0
for i in range(len(cluster_table)):
    if cluster_table['isMultiple'][i] == 1.0:
        first_binary = i
        break
print(first_binary)

compMasses = []
for i in range(len(cluster_table)):
    if cluster_table['isMultiple'][i] == 1.0:
        compMasses.append(cluster_table['compMass'][i][0])
print(compMasses)

1745
[0.018772196398952473, 0.020398101994065967, 0.04970394506596586, 0.011418578712969283, 0.1109845537581072, 0.08288741359987532, 0.054993759410529613, 0.33087717935666183, 0.17271523389867505, 0.013492928385704234, 0.06711686707284298, 0.10798400551211244, 0.029492553417922056, 0.14367710080576154, 0.26903433005844346, 0.08474773851417283, 0.30095260930078693, 0.05497012322155322, 0.09173333630199015, 0.025891768717209534, 0.02735674785778421, 0.1558841429805367, 0.048604116260690874, 0.04128694062892624, 0.03487493039755069, 0.14196306235013237, 0.04280274224495466, 0.02310867140421406, 0.3955789882313527, 0.15676043958471955, 0.08859345991063905, 0.055069286248966655, 0.1552959762503858, 0.09515570587436127, 0.04718531929120123, 0.03205825521948448, 0.011501610127550384, 1.1694443225860596, 0.0922981857375905, 0.11274085938930511, 0.16108334558162823, 0.07456852106262159, 0.013275199569761753, 0.11918815425644891, 0.038134801276114384, 0.019446755405924193, 0.10889488258261569, 

In [6]:
start = (min(level_ages) // 0.5e6) * 0.5e6
end = (max(level_ages) // 0.5e6) * 0.5e6 + 2 * 0.5e6
increment = 0.5e6  # 500,000

# Create age arrays
level_age_arr = np.arange(start, end, increment)
log_age_arr = np.log10(level_age_arr)

print(log_age_arr)
print(level_age_arr)

[6.39794001 6.47712125 6.54406804 6.60205999 6.65321251 6.69897
 6.74036269 6.77815125 6.81291336 6.84509804 6.87506126 6.90308999
 6.92941893 6.95424251 6.97772361 7.         7.0211893  7.04139269
 7.06069784 7.07918125 7.09691001 7.11394335 7.13033377 7.14612804
 7.161368   7.17609126 7.1903317  7.20411998 7.21748394 7.23044892
 7.24303805 7.25527251 7.26717173 7.2787536  7.29003461 7.30103
 7.31175386]
[ 2500000.  3000000.  3500000.  4000000.  4500000.  5000000.  5500000.
  6000000.  6500000.  7000000.  7500000.  8000000.  8500000.  9000000.
  9500000. 10000000. 10500000. 11000000. 11500000. 12000000. 12500000.
 13000000. 13500000. 14000000. 14500000. 15000000. 15500000. 16000000.
 16500000. 17000000. 17500000. 18000000. 18500000. 19000000. 19500000.
 20000000. 20500000.]


In [ ]:
# create isochrone grid - if this is the first time, then this is going to take an hour lmfao
instances = np.empty(len(log_age_arr), dtype=object)

for i in range(len(log_age_arr)):
    my_iso = synthetic.IsochronePhot(log_age_arr[i], AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
    instances[i] = my_iso
        
print("isochrone generation done")

Isochrone generation took 1.131738 s.
Making photometry for isochrone: log(t) = 6.40  AKs = 0.00  dist = 410
     Starting at:  2023-12-19 04:08:48.583305   Usually takes ~5 minutes
Starting filter: jwst,F115W   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.070 Msun  T =  2904 K  m_jwst_F115W = 14.69
Starting filter: jwst,F182M   Elapsed time: 0.44 seconds
Starting synthetic photometry
M =   0.070 Msun  T =  2904 K  m_jwst_F182M = 13.95
      Time taken: 0.88 seconds
Isochrone generation took 0.931034 s.
Making photometry for isochrone: log(t) = 6.48  AKs = 0.00  dist = 410
     Starting at:  2023-12-19 04:08:50.414208   Usually takes ~5 minutes
Starting filter: jwst,F115W   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.070 Msun  T =  2906 K  m_jwst_F115W = 14.78
Starting filter: jwst,F182M   Elapsed time: 0.44 seconds
Starting synthetic photometry
M =   0.070 Msun  T =  2906 K  m_jwst_F182M = 14.04
      Time taken: 0.87 seconds
Isochrone generati

      Time taken: 0.88 seconds
Isochrone generation took 0.867254 s.
Making photometry for isochrone: log(t) = 7.04  AKs = 0.00  dist = 410
     Starting at:  2023-12-19 04:09:18.786832   Usually takes ~5 minutes
Starting filter: jwst,F115W   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.070 Msun  T =  2945 K  m_jwst_F115W = 16.29
Starting filter: jwst,F182M   Elapsed time: 0.43 seconds
Starting synthetic photometry
M =   0.070 Msun  T =  2945 K  m_jwst_F182M = 15.52
      Time taken: 0.88 seconds
Isochrone generation took 0.852528 s.
Making photometry for isochrone: log(t) = 7.06  AKs = 0.00  dist = 410
     Starting at:  2023-12-19 04:09:20.528232   Usually takes ~5 minutes
Starting filter: jwst,F115W   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.070 Msun  T =  2945 K  m_jwst_F115W = 16.33
Starting filter: jwst,F182M   Elapsed time: 0.43 seconds
Starting synthetic photometry
M =   0.070 Msun  T =  2945 K  m_jwst_F182M = 15.56
      Time taken: 

Starting filter: jwst,F182M   Elapsed time: 0.44 seconds
Starting synthetic photometry
M =   0.070 Msun  T =  2940 K  m_jwst_F182M = 15.99
      Time taken: 0.88 seconds
Isochrone generation took 0.871694 s.
Making photometry for isochrone: log(t) = 7.29  AKs = 0.00  dist = 410
     Starting at:  2023-12-19 04:09:48.526309   Usually takes ~5 minutes
Starting filter: jwst,F115W   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.070 Msun  T =  2940 K  m_jwst_F115W = 16.80
Starting filter: jwst,F182M   Elapsed time: 0.44 seconds
Starting synthetic photometry
M =   0.070 Msun  T =  2940 K  m_jwst_F182M = 16.03
      Time taken: 0.87 seconds


In [ ]:
# identify array indeces, create array for labels and colors
idx_arr = range(0, len(log_age_arr))
if len(idx_arr) > 3:
    idx_arr = idx_arr[::2]

# Plot CMD
fig, axes = py.subplots(figsize=(15, 10))
py.subplot(1, 2, 1)
for i in range(len(idx_arr)):
    idx = idx_arr[i]
    if i == 0:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(idx_arr) - 1:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'red', label='')
    
py.xlabel('F115W - F182M')
py.ylabel('F182M')
py.gca().invert_yaxis()
py.legend()

In [ ]:
import sys
sys.path.append('/scratch/wyz5rge/synthetic-hr/12-18')
import interpolatorExp as interpolator

level_ages_myr = level_ages / 1e6
# print(level_ages_myr)

# create array of stars
stars = np.empty(len(cluster_table), dtype=object)

# perform interpolation for each star
for i in range(len(stars)):
    # print(str(i) + " " + str(level_ages_myr[i]) + " " + str(masses[i]))
    stars[i] = interpolator.interpolate(level_ages_myr[i], masses[i], instances, log_age_arr, filters)
    
# convert luminosity values to solar luminosities
watts_to_lsun = 1.0 / (3.846e26) # conversion factor for watts to Lsun

for i in range(len(stars)):
    if stars[i] is None:
        continue
    stars[i][0] = stars[i][0] * watts_to_lsun
    
for i in range(len(stars)):
    print(stars[i])

In [ ]:
fig, axes = py.subplots(figsize=(15, 10))

# plot CMD
py.subplot(1, 2, 1)
for i in range(len(idx_arr)):
    idx = idx_arr[i]
    if i == 0:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(idx_arr) - 1:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'red', label='')
    
py.xlabel('F115W - F182M')
py.ylabel('F182M')
py.gca().invert_yaxis()
py.legend()

for i in range(len(star)):
    if stars[i] is None:
        continue
    py.plot(stars[i][3] - stars[i][4], stars[i][4], marker='o', markersize=1, color='k')
    
for i in range(len(unresolved_binaries)):
    if unresolved_binaries[i] is None:
        continue
    py.plot(unresolved_binaries[i][0] - unresolved_binaries[i][1], unresolved_binaries[i][1], marker='o', markersize=1, color='k')